In [1]:
# Setting up a model and a mesh for the MT forward problem

In [6]:
import SimPEG as simpeg, sys
import numpy as np
from SimPEG import NSEM
sys.path.append('/home/gudni/gitCodes/python/telluricpy')
import telluricpy

In [52]:
# Define the area of interest
bw, be = 557100, 557580
bs, bn = 7133340, 7133960
bb, bt = 0,480

In [27]:
# Build the mesh
# Design the tensors
hSize,vSize =  20., 10
nrCcore = [15, 8, 6, 5, 4, 2, 2, 2, 2]
hPad = simpeg.Utils.meshTensor([(hSize,10,1.5)])
hx = np.concatenate((hPad[::-1],np.ones(((be-bw)/hSize,))*hSize,hPad))
hy = np.concatenate((hPad[::-1],np.ones(((bn-bs)/hSize,))*hSize,hPad))
airPad = simpeg.Utils.meshTensor([(vSize,13,1.5)])
vCore = np.concatenate([ np.ones(i)*s for i, s in zip(nrCcore,(simpeg.Utils.meshTensor([(vSize,1),(vSize,8,1.3)])))])[::-1]
botPad = simpeg.Utils.meshTensor([(vCore[0],8,-1.5)])
hz = np.concatenate((botPad,vCore,airPad))
# Calculate the x0 point
x0 = np.array([bw-np.sum(hPad),bs-np.sum(hPad),bt-np.sum(vCore)-np.sum(botPad)])
# Make the mesh
meshFor = simpeg.Mesh.TensorMesh([hx,hy,hz],x0)


In [28]:
print np.sum(vCore)
print meshFor.nC
print meshFor

1038.9319582
150348
  ---- 3-D TensorMesh ----  
   x0: 553700.10
   y0: 7129940.10
   z0: -6586.10
  nCx: 44
  nCy: 51
  nCz: 67
   hx: 1153.30, 768.87, 512.58, 341.72, 227.81, 151.88, 101.25, 67.50, 45.00, 30.00, 24*20.00, 30.00, 45.00, 67.50, 101.25, 151.88, 227.81, 341.72, 512.58, 768.87, 1153.30
   hy: 1153.30, 768.87, 512.58, 341.72, 227.81, 151.88, 101.25, 67.50, 45.00, 30.00, 31*20.00, 30.00, 45.00, 67.50, 101.25, 151.88, 227.81, 341.72, 512.58, 768.87, 1153.30
   hz: 2090.63, 1393.75, 929.17, 619.45, 412.96, 275.31, 183.54, 122.36, 2*81.57, 2*62.75, 2*48.27, 2*37.13, 4*28.56, 5*21.97, 6*16.90, 8*13.00, 15*10.00, 15.00, 22.50, 33.75, 50.62, 75.94, 113.91, 170.86, 256.29, 384.43, 576.65, 864.98, 1297.46, 1946.20


In [29]:
# Save the mesh
meshFor.writeVTK('nsmesh_FineHKPK1.vtr',{'id':np.arange(meshFor.nC)})
nsvtr = telluricpy.vtkTools.io.readVTRFile('nsmesh_FineHKPK1.vtr')

In [30]:
topoSurf = telluricpy.vtkTools.polydata.normFilter(telluricpy.vtkTools.io.readVTPFile('../Geological_model/CDED_Lake_Coarse.vtp'))
activeMod = telluricpy.vtkTools.extraction.extractDataSetWithPolygon(nsvtr,topoSurf)

In [31]:
telluricpy.vtkTools.io.writeVTUFile('activeModel.vtu',activeMod)

In [32]:
# Get active indieces 
activeInd = telluricpy.vtkTools.dataset.getDataArray(activeMod,'id')

In [37]:
# Make the conductivity dictionary
# Note: using the background value for the till, since the extraction gets the ind's below the till surface
geoStructFileDict = {'Till':1e-4,
 'PK1':5e-2,
 'HK1':1e-3,
  'VK':5e-3}

In [38]:
# Loop through
extP = '../Geological_model/'
geoStructIndDict = {}
for key, val in geoStructFileDict.iteritems():
    geoPoly = telluricpy.vtkTools.polydata.normFilter(telluricpy.vtkTools.io.readVTPFile(extP+key+'.vtp'))
    modStruct = telluricpy.vtkTools.extraction.extractDataSetWithPolygon(activeMod,geoPoly,extBoundaryCells=True,extInside=True,extractBounds=True)
    geoStructIndDict[key] = telluricpy.vtkTools.dataset.getDataArray(modStruct,'id')

In [39]:
# Make the physical prop
sigma = np.ones(meshFor.nC)*1e-8
sigma[activeInd] = 1e-3 # 1e-4 is the background and 1e-3 is the till value
# Add the structure
for key in ['Till','PK1','HK1','VK']:
    sigma[geoStructIndDict[key]] = geoStructFileDict[key]

In [40]:
# Save the model
meshFor.writeVTK('nsmesh_FineHKPK1Fin.vtr',{'S/m':sigma})

In [42]:
# Set up the forward modeling
freq = np.logspace(5,2,16)
np.save('MTfrequencies',freq)

In [67]:
# Find the locations on the surface of the model.
# Get the outer shell of the model
import vtk
actModVTP = telluricpy.vtkTools.polydata.normFilter(telluricpy.vtkTools.extraction.geometryFilt(activeMod))
polyBox = vtk.vtkCubeSource()
polyBox.SetBounds(bw-5.,be+5,bs-5.,bn+5.,bb-5.,bt+5)
polyBox.Update()
# Exract the topo of the model
modTopoVTP = telluricpy.vtkTools.extraction.extractDataSetWithPolygon(actModVTP,telluricpy.vtkTools.polydata.normFilter(polyBox.GetOutput()),extractBounds=False)
telluricpy.vtkTools.io.writeVTPFile('topoSurf.vtp',actModVTP)

In [68]:
# Make the rxLocations file
x,y = np.meshgrid(np.arange(bw+10.,be,20),np.arange(bs+10.,bn,20))
xy = np.hstack((x.reshape(-1,1),y.reshape(-1,1)))
# Find the location array
locArr = telluricpy.modelTools.surfaceIntersect.findZofXYOnPolydata(xy,actModVTP) #modTopoVTP)
np.save('MTlocations',locArr)

In [70]:
telluricpy.vtkTools.io.writeVTPFile('MTloc.vtp',telluricpy.dataFiles.XYZtools.makeCylinderPtsVTP(locArr,5,10,10))

In [60]:
# Running the forward modelling on the Cluster.
# Define the forward run in findDiam_MTforward.py

In [71]:
%matplotlib qt
sys.path.append('/home/gudni/Dropbox/code/python/MTview/')
import interactivePlotFunctions as iPf

In [72]:
# Load the data
mtData = np.load('MTdataStArr_nsmesh_HKPK1.npy')
mtData

array([ (100000.0, 557110.0, 7133350.0, 430.0, (-1.1506946677168877-1.2968733538942032j), (-23.54311070144429-19.772598506113592j), (23.122674523601596+16.553191376249956j), (0.9829648263716114+1.0160859204765444j), (-0.01149115639336881-0.08206248665420915j), (-0.0456173506763778-0.034164268293959334j)),
       (100000.0, 557130.0, 7133350.0, 430.0, (-0.7746917073255672-1.159486343238143j), (-22.964390132774177-20.1959003955084j), (21.4348641328774+16.38053800316572j), (0.6560327531217727+0.9550969901843831j), (-0.02089376870480284-0.06961905697131537j), (-0.053884654330991905-0.04363080750086569j)),
       (100000.0, 557150.0, 7133350.0, 430.0, (-0.6752011886313354-1.1676037152435292j), (-22.491532296418313-20.52581540138984j), (20.413657171072817+16.42466481134132j), (0.6638087171680884+1.0822899762152727j), (-0.026225671652060738-0.057567627605059984j), (-0.06032865752192971-0.05272531832396195j)),
       ...,
       (100.0, 557530.0, 7133950.0, 440.0, (0.44311335566511817+0.135906

In [110]:
iPf.MTinteractiveMap([mtData])

In [104]:
# Looking at the data shows that data below 100Hz is affected by the boundary conditions, 
# which makes sense for very conductive conditions as we have.
# Invert data in the 1e5-1e2 range.

Run the inversion on the cluster using the inv3d/run1/findDiam_inversion.py

In [106]:
drecAll = np.load('MTdataStArr_nsmesh_0.npy')

In [109]:
np.unique(drecAll['freq'])[10::]

array([    100.        ,     158.48931925,     251.18864315,
           398.10717055,     630.95734448,    1000.        ,
          1584.89319246,    2511.88643151,    3981.07170553,
          6309.5734448 ,   10000.        ,   15848.93192461,
         25118.8643151 ,   39810.71705535,   63095.73444802,  100000.        ])

In [93]:
# Build the Inversion mesh
# Design the tensors
hSizeI,vSizeI =  25., 10.
nrCcoreI = [12, 6, 6, 6, 5, 4, 3, 2, 1]
hPadI = simpeg.Utils.meshTensor([(hSizeI,9,1.5)])
hxI = np.concatenate((hPadI[::-1],np.ones(((be-bw)/hSizeI,))*hSizeI,hPadI))
hyI = np.concatenate((hPadI[::-1],np.ones(((bn-bs)/hSizeI,))*hSizeI,hPadI))
airPadI = simpeg.Utils.meshTensor([(vSizeI,12,1.5)])
vCoreI = np.concatenate([ np.ones(i)*s for i, s in zip(nrCcoreI,(simpeg.Utils.meshTensor([(vSizeI,1),(vSizeI,8,1.3)])))])[::-1]
botPadI = simpeg.Utils.meshTensor([(vCoreI[0],7,-1.5)])
hzI = np.concatenate((botPadI,vCoreI,airPadI))
# Calculate the x0 point
x0I = np.array([bw-np.sum(hPadI),bs-np.sum(hPadI),bt-np.sum(vCoreI)-np.sum(botPadI)])
# Make the mesh
meshInv = simpeg.Mesh.TensorMesh([hxI,hyI,hzI],x0I)

In [94]:
meshInv.writeVTK('nsmesh_HPVK1_inv.vtr',{'id':np.arange(meshInv.nC)})

In [101]:
nsInvvtr = telluricpy.vtkTools.io.readVTRFile('nsmesh_HPVK1_inv.vtr')
activeModInv = telluricpy.vtkTools.extraction.extractDataSetWithPolygon(nsInvvtr,topoSurf,extBoundaryCells=True)

In [102]:
sigma = np.ones(meshInv.nC)*1e-8
indAct = telluricpy.vtkTools.dataset.getDataArray(activeModInv,'id')
sigma[indAct] = 1e-4

In [103]:
meshInv.writeVTK('nsmesh_HPVK1_inv.vtr',{'id':np.arange(meshInv.nC),'S/m':sigma})

In [108]:
from pymatsolver import MumpsSolver

ImportError: cannot import name MumpsSolver

In [106]:
pymatsolver.AvailableSolvers

{'DiagonalSolver': True,
 'Mumps': False,
 'TriangleFortran': False,
 'TrianglePython': True}

In [116]:
NSEM.Utils.skindepth(1000,100000)

50.329212104487041

In [117]:
np.unique(mtData['freq'])[5::]

array([   1000.        ,    1584.89319246,    2511.88643151,
          3981.07170553,    6309.5734448 ,   10000.        ,
         15848.93192461,   25118.8643151 ,   39810.71705535,
         63095.73444802,  100000.        ])